In [12]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense,GRU
from sklearn.preprocessing import MinMaxScaler
from keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Bidirectional
import warnings
warnings.filterwarnings('ignore')


data_clean = pd.read_csv("future_ss2312_tick.csv")


# 1. 数据加载和预处理
# 确保'close'列是数值型
data_clean['close'] = pd.to_numeric(data_clean['close'], errors='coerce')

price_features = ['昨收盘', '今开盘', '最高价', '最低价', '申买价一', '申卖价一']
for feature in price_features:
    data_clean[feature + '_diff'] = data_clean['close'] - data_clean[feature]

data_clean['trade_time'] = pd.to_datetime(data_clean['trade_time'])

# 将时间分解为小时和分钟
data_clean['hour'] = data_clean['trade_time'].dt.hour
data_clean['minute'] = data_clean['trade_time'].dt.minute

data_clean['close_diff'] = data_clean['close'].diff()

# Define label
data_clean['label'] = (data_clean['close'].shift(-100) > data_clean['close']).astype(int)

features = ['close_diff', '数量', 'hour', 'minute'] + [f + '_diff' for f in price_features]

# 3. 分割数据

# Now you can filter the data between two dates
train_data = data_clean[(data_clean['trade_time'] >= '2023-09-01 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-12 09:00:00')]

test_data = data_clean[(data_clean['trade_time'] >= '2023-10-12 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-20 09:00:00')]


# 初始化归一化器
scaler = MinMaxScaler(feature_range=(0, 1))

train_data[features] = scaler.fit_transform(train_data[features])


# 将 DataFrame 转换为 NumPy 数组
X_train = np.array(train_data[features])
y_train = np.array(train_data['label'])

# 删除 NaN 值
mask = ~np.isnan(X_train).any(axis=1)
X_train = X_train[mask]
y_train = y_train[mask]

# 首先，确保 X_train 和 X_test 没有 NaN 值
X_train = X_train[~np.isnan(X_train).any(axis=1)]
y_train = y_train[~np.isnan(X_train).any(axis=1)]

class TimeseriesGenerator(Sequence):
    def __init__(self, data, labels, length, stride=1, batch_size=32):
        self.data = data
        self.labels = labels
        self.length = length
        self.stride = stride
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil((len(self.data) - self.length) / float(self.stride * self.batch_size)))

    def __getitem__(self, idx):
        batch_x = []
        batch_y = []

        start = idx * self.batch_size * self.stride
        end = start + self.batch_size * self.stride + self.length

        for i in range(start, min(end, len(self.data) - self.length), self.stride):
            batch_x.append(self.data[i: i + self.length])
            batch_y.append(self.labels[i + self.length])

        return np.array(batch_x), np.array(batch_y)

# 定义时间步长和步长
time_steps = 200
stride = 1  # 增加步长以减少内存使用

# 创建数据生成器
train_generator = TimeseriesGenerator(X_train, y_train, length=time_steps, stride=stride, batch_size=32)

# 创建 LSTM 模型
model = Sequential()
model.add(GRU(units=50, input_shape=(time_steps, X_train.shape[1])))
# model.add(Dropout(0.2))  # Dropout层
model.add(Dense(1, activation='sigmoid'))  # 输出层，适用于二分类任务
model.compile(optimizer='adam', loss='binary_crossentropy')

# 使用生成器训练模型
model.fit_generator(train_generator, epochs=5)
from keras.models import load_model
model.save('model_gru.h5')



Epoch 1/5


2023-11-22 10:28:28.679192: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


19136/19136 [==============================] - 2475s 129ms/step - loss: 0.6145
Epoch 2/5
19136/19136 [==============================] - 3319s 173ms/step - loss: 0.6021
Epoch 3/5
19136/19136 [==============================] - 3355s 175ms/step - loss: 0.5977
Epoch 4/5
19136/19136 [==============================] - 3156s 165ms/step - loss: 0.5929
Epoch 5/5
19136/19136 [==============================] - 3607s 189ms/step - loss: nan


In [13]:
from keras.models import load_model
model.save('model_gru.h5')

In [ ]:
def predict_next_move(tick, model, time_steps,historical_data,scaler):
    # 将新的 tick 数据追加到历史数据中
    historical_data = pd.concat([historical_data, pd.DataFrame([tick])], ignore_index=True)
    

    # 检查是否有足够的数据来计算滚动和EWM特征
    if len(historical_data) >= time_steps+20:


        for feature in price_features:
            historical_data[feature + '_diff'] = historical_data['close'] - historical_data[feature]

        # 将时间分解为小时和分钟
        historical_data['hour'] = historical_data['trade_time'].dt.hour
        historical_data['minute'] = historical_data['trade_time'].dt.minute

        historical_data['close_diff'] = historical_data['close'].diff()


        data_for_scaling = historical_data[features].dropna()

        # 选择最近的time_steps行用于归一化
        data_to_scale = data_for_scaling.tail(time_steps)

        # 归一化
        scaled_data = scaler.transform(data_to_scale)
        

        # 使用归一化的数据创建模型输入
        X_new = scaled_data.reshape(1, time_steps, len(features))


        # 检查X_new是否包含NaN值
        if np.isnan(X_new).any():
            return None, historical_data
        else:
            # 进行预测
            prediction_proba = model.predict(X_new,verbose=0)
            probability_of_one = prediction_proba[0][0]

            return probability_of_one, historical_data
    else:
        # 数据不足以进行预测
        return None, historical_data





# Initialize historical_data with the correct column names and types if necessary
historical_data = pd.DataFrame()
scaled_historical_data = pd.DataFrame()

initial_funds = 100000
funds = initial_funds
stock_quantity = 0
stock_price = 0
buy_threshold = 0.6
sold_threshold = 0.4
transactions = []
minute_count = 0

for idx, row in test_data.iterrows():
    current_probability, historical_data = predict_next_move(row, model,time_steps,historical_data,scaler) 
    # print(current_probability)
    # print(row['trade_time'])
    if current_probability is not None:
        current_price = row['close']
        minute_count = minute_count + 1
            
        if current_probability > buy_threshold and stock_quantity == 0 :
            print(current_probability)
            stock_quantity = funds // current_price
            funds -= stock_quantity * current_price
            fee = stock_quantity * 2
            funds -= fee
            stock_price = current_price
            buy_price = current_price  # 记录买入价格
            print(f"Time: {row['trade_time']} - Action: BUY at {current_price}, Quantity: {stock_quantity},Funds:{funds}")
            transactions.append({
                'action': 'buy',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds
            })
            minute_count = 0
        elif minute_count > 300 and stock_quantity > 0 and current_probability<sold_threshold:
            print(current_probability)
            funds += stock_quantity * current_price
            price_diff = current_price - buy_price  # 计算价格差异
            # 判断价格差异是涨、跌还是平
            if price_diff > 0:
                direction = '涨'
            elif price_diff < 0:
                direction = '跌'
            else:
                direction = '平'
            print(f"Time: {row['trade_time']} - Action: SELL at {current_price}, Quantity: {stock_quantity},Funds:{funds}, Price Change: {direction}")
            transactions.append({
                'action': 'sell',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds,
                'price_change': direction
            })
            stock_quantity = 0
            buy_price = 0  # 重置买入价格为0

    
if len(transactions)>0:
    transactions_df = pd.DataFrame(transactions)
    print(transactions_df['price_change'].value_counts())
transactions_df.to_csv('transactions_tick.csv')

In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.models import load_model

# 假设这些变量已经在上下文中定义
# model = load_model('gru_model.h5')
time_steps = 200
price_features = ['昨收盘', '今开盘', '最高价', '最低价', '申买价一', '申卖价一']
features = ['close_diff', '数量', 'hour', 'minute'] + [f + '_diff' for f in price_features]
scaler = MinMaxScaler(feature_range=(0, 1))

def prepare_data_for_prediction(test_data, time_steps, scaler):

    # 使用归一化
    scaled_data = scaler.fit_transform(test_data[features].dropna())

    # 重塑数据以适应模型
    X = np.array([scaled_data[i:i+time_steps] for i in range(len(scaled_data)-time_steps+1)])
    return X

# 准备数据
X_test = prepare_data_for_prediction(test_data, time_steps, scaler)
# 批量预测
predictions = model.predict(X_test, verbose=0)
# 确保test_data的索引与predictions对齐
aligned_test_data = test_data.iloc[time_steps - 1:]


initial_funds = 100000
funds = initial_funds
stock_quantity = 0
stock_price = 0
buy_threshold = 0.7
sold_threshold = 0.4
transactions = []
minute_count = 0
for prediction,  (index, row)  in zip(predictions, aligned_test_data.iterrows()):
    current_probability = prediction[0]
    if current_probability is not None:
        current_price = row['close']
        minute_count = minute_count + 1
            
        if current_probability > buy_threshold and stock_quantity == 0 :
            print(current_probability)
            stock_quantity = funds // current_price
            funds -= stock_quantity * current_price
            fee = stock_quantity * 2
            funds -= fee
            stock_price = current_price
            buy_price = current_price  # 记录买入价格
            print(f"Time: {row['trade_time']} - Action: BUY at {current_price}, Quantity: {stock_quantity},Funds:{funds}")
            transactions.append({
                'action': 'buy',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds
            })
            minute_count = 0
        elif minute_count > 200 and stock_quantity > 0 and current_probability<sold_threshold:
            print(current_probability)
            funds += stock_quantity * current_price
            price_diff = current_price - buy_price  # 计算价格差异
            # 判断价格差异是涨、跌还是平
            if price_diff > 0:
                direction = '涨'
            elif price_diff < 0:
                direction = '跌'
            else:
                direction = '平'
            print(f"Time: {row['trade_time']} - Action: SELL at {current_price}, Quantity: {stock_quantity},Funds:{funds}, Price Change: {direction}")
            transactions.append({
                'action': 'sell',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds,
                'price_change': direction
            })
            stock_quantity = 0
            buy_price = 0  # 重置买入价格为0

    
if len(transactions)>0:
    transactions_df = pd.DataFrame(transactions)
    print(transactions_df['price_change'].value_counts())
transactions_df.to_csv('transactions_tick_gru.csv')